In [25]:
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

# Convert water sources

Read in water sources (CSV) and convert to `pumps.json` file used in the [Münster schenkt aus](https://github.com/digifarm-ms/giessdenkiez-de) web application.

(1) Read input data (with correct character encoding and delimiter):

In [26]:
df = pd.read_csv("./input/Wasserstandorte2023.csv", encoding='iso-8859-1', delimiter=';')
df.columns = ['id', 'district', 'address', 'coordinates']
df.head()

,id,district,address,coordinates
0,1,Albachten,Albachtener Park,"51.918638, 7.521297"
1,2,Roxel,Schwerinerstr 25 (Art Parkbucht),"51.952700, 7.543774"
2,3,Gievenbeck,Dieckmannstr. 6,"51.972357,7.564990"
3,4,Gievenbeck,Dieckmannstr. 131 (Höhe Mosaikschule),"51.964894,7.566887"
4,5,Gievenbeck,Toppheideweg Platz,"51.975344, 7.569535"


(2) Extract latitude and longitude values from the `coordinates` column

In [27]:
df[['lat', 'lon']] = df['coordinates'].str.split(',', expand=True)
df[['address', 'lon', 'lat']].head()


,address,lon,lat
0,Albachtener Park,7.521297,51.918638
1,Schwerinerstr 25 (Art Parkbucht),7.543774,51.952700
2,Dieckmannstr. 6,7.564990,51.972357
3,Dieckmannstr. 131 (Höhe Mosaikschule),7.566887,51.964894
4,Toppheideweg Platz,7.569535,51.975344


(3) Create GeoJSON feature collection from the data:

In [28]:
features = df.apply(
    lambda row: Feature(
        geometry=Point((float(row['lon']), float(row['lat']))),
        properties={
            'id': row['id'],
            'addr:full': row['address'],
        }
    ),
    axis=1).tolist()

feature_collection = FeatureCollection(features)
print(json.dumps(feature_collection, indent=2))


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          7.521297,
          51.918638
        ]
      },
      "properties": {
        "id": 1,
        "addr:full": "Albachtener Park"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          7.543774,
          51.9527
        ]
      },
      "properties": {
        "id": 2,
        "addr:full": "Schwerinerstr 25 (Art Parkbucht)"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          7.56499,
          51.972357
        ]
      },
      "properties": {
        "id": 3,
        "addr:full": "Dieckmannstr. 6"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          7.566887,
          51.964894
        ]
      },
      "properties

(4) Export GeoJSON data:

In [29]:
with open('./output/pumps.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, indent=2, ensure_ascii=False)


(5) Export GeoJSON data without indentation (minified):

In [30]:
with open('./output/pumps.geojson.min.json', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)
